## DAT19 Lab 07
## Logistic Regression Classification

Import the usual packages

In [ ]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure,show,output_notebook
from bokeh.models import Range1d

from sklearn import datasets
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

output_notebook()
%matplotlib inline

Here are a couple pandas settings to make viewing the data a little easier

In [ ]:
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',60)

#### Load the Iris Data Set

In [ ]:
from sklearn import datasets

sk_iris = datasets.load_iris()
iris = pd.DataFrame(sk_iris.data,columns=sk_iris['feature_names'])
iris['target'] = sk_iris.target
Names = sk_iris.target_names

#### Logistic Regression is a binary classifier so we'll just use two classes of the data set

In [ ]:
iris = iris[iris.target!=0]

In [ ]:
iris.head()

This is the standard cross validation function we can use to get a score on our models

In [ ]:
from sklearn.cross_validation import cross_val_score

### Create an instance of a Logistic Regression model and apply cross-validation
#### (Note: C parameter is for regularization, also known as our complexity penalty)

In [ ]:
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(C=1)

In [ ]:
features = iris.drop('target',axis=1)
target = iris.target

In [ ]:
cross_val_score(model_lr,features,target,cv=3).mean()

### Review the feature importance


In [ ]:
model_lr = LogisticRegression(C=1).fit(features, target)

x = np.arange(len(features.columns))
names = features.columns
names

OR = e^Beta and the betas are the model coefficients.

In [ ]:
print model_lr.coef_
print model_lr.coef_.ravel()

In [ ]:
p = figure(title="Model Coefficients")
for val in x:
    p.quad(top=model_lr.coef_.ravel()[val], 
           bottom=0, left=val+0.2,right=val+0.8, 
           color=['red','orange','green','purple'][val],
           legend=names[val]
          )
    
p.y_range = Range1d(min(model_lr.coef_.ravel())-0.1, max(model_lr.coef_.ravel())+1.5)
show(p)


#### Exercise 1: Try Changing the Normalization and see how the Coeficients Change and Discuss with your Neighbor

## Example 2: Build Logistic Regression Classifier for Spambase Data

#### Load the spambase.csv as a pandas DataFrame (last column of data contains Target Data - is_spam)

In [ ]:
spam_data = pd.read_csv("../data/spambase.csv")
# spam_data.head()
spam_data.describe()
# spam_data.info()

Separate features from the rest of the data to use in our algorithms

In [ ]:
features = spam_data.drop('is_spam',axis=1)
target = spam_data.is_spam

# features.head()
target.head()

Convert DataFrames into numpy arrays

In [ ]:
features.values

#### Use crossvalidation to score model

In [ ]:
from sklearn.linear_model import LogisticRegression

features = spam_data.drop('is_spam',axis=1)
target = spam_data.is_spam

model_lr = LogisticRegression(C=1)

In [ ]:
cross_val_score(model_lr,features,target,cv=3).mean()

### Compare Performance of Logistic Regression to KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(3)
cross_val_score(model_knn,features,target,cv=3).mean()

#### Exercise 2: Try testing and plotting CV Scores with different C values

In [ ]:
from sklearn.linear_model import LogisticRegression

c_list = [1000, 100, 10, 1, 0.1, 0.01, 0.001, .0001]
cv_scores = []


###  Evaluate Feature Importance

In [ ]:
feature_names = spam_data.columns[:-1]
feature_names = feature_names[-10:]
model_lr = LogisticRegression(C=1).fit(features, target)



p = figure(title="Model Coefficients")

coefficients=model_lr.coef_.ravel()[-10:]

x = np.arange(len(feature_names))
for val in x:
    p.quad(top = coefficients[val], bottom=0, left=val+0.2,
           right=val+0.8, color=['red','orange','green','purple','blue','cyan','magenta','red','orange','purple','blue'][val],legend=feature_names[val])
p.y_range = Range1d(min(coefficients)-0.1, max(coefficients)+1.5)
show(p)


### What if I want the most important features?

In [ ]:
coeffs = pd.DataFrame(zip(spam_data.columns[:-1],model_lr.coef_.ravel()),columns=['features','coeff'])
coeffs.head()

In [ ]:
coeffs['abs'] = np.absolute(coeffs.coeff.values)

coeffs.sort('abs',ascending=False)

### Data Normalization

In [ ]:
features.describe()

#### Normalize all features in one line of code
This is the form of normalization where you normalize the standard deviation.

In [ ]:
features_norm = (features - features.mean())/features.std()

In [ ]:
features_norm.describe()

We can also use the built in SKLearn Normalization

In [ ]:
scaler = StandardScaler()
features_norm_2 = scaler.fit_transform(features)

pd.DataFrame(features_norm_2, columns=features.columns).head()

In [ ]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(C=1)

cross_val_score(model_lr,features_norm, target,cv=3 ).mean()

#### Chart last 10 features and compare with earlier coefficients

In [ ]:
feature_names = spam_data.columns[:-1]

feature_names = feature_names[-11:]
feat_norm_val = features_norm.values
model_lr = LogisticRegression(C=1).fit(feat_norm_val, target)

p = figure(title="Model Coefficients")

coefficients=model_lr.coef_.ravel()[-11:]

x = np.arange(len(feature_names))
for val in x:
    p.quad(top = coefficients[val], bottom=0, left=val+0.2,
           right=val+0.8, 
           color=['red','orange','green','purple','blue','cyan','magenta','red','orange','purple','blue','green'][val],
           legend=feature_names[val])
p.y_range = Range1d(min(coefficients)-0.1, max(coefficients)+2.5)
show(p)


### Exercise 3:
Using what we know in a real life example:

Bank Marketing!

Load the bank.csv file and using the column `y` as your target, determine feature importance of convincing someone to buy a term deposit. For more background: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

Outcomes:
1. Identify most predictive features and explain why they do (or do not?) make sense.
2. Discuss with your neighbor which features you could use in crafting an outreach plan for marketing purposes.
3. What would you tell the Chief Marketing Officer? Can you quantify the improvement?

In [ ]:
bank = pd.read_csv("../data/bank.csv")
# bank.head()
bank.info()

HINT: Convert the columns that are binary to 0 or 1 using .map()

In [ ]:
#Define a dictionary that will map a value to a string in a pandas column
married_mapping = {'married':0,'single':1,'divorced':1}
# use .map() on the column
bank['marital']=bank['marital'].map(married_mapping)
bank['marital']